# 1.Data Extraction & Download



## 1.1 Data import

In [1]:
# @title
try:
  from vnstock import Vnstock
except:
  !pip install -U vnstock
  from vnstock import Vnstock
import pandas as pd
import numpy as np
import datetime as dt
import time
import statsmodels.api as sm
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from collections import defaultdict
from google.colab import files
from collections import defaultdict
from scipy.stats import ttest_1samp
from IPython.display import display
import matplotlib.ticker as mtick


try:
  from linearmodels import PanelOLS
except:
  !pip install linearmodels
  from linearmodels import PanelOLS


📋 Connecting Google Drive account
to save project settings.

Mounted at /content/drive


In [2]:
# @title
def load_excel(filename, sheet_name=0, header=0):
    try:
        return pd.read_excel(filename, sheet_name=sheet_name, header=header, engine="openpyxl")
    except:
        print(f"--- Import file: {filename} ---")
        try:
            files.upload()
            return pd.read_excel(filename, sheet_name=sheet_name, header=header, engine="openpyxl")
        except:
            !pip install openpyxl
            files.upload()
            return pd.read_excel(filename, sheet_name=sheet_name, header=header, engine="openpyxl")


# df_BS      = load_excel("vietnam_stocks_balance_sheets.xlsx")
# df_ratio   = load_excel("vietnam_stocks_ratios.xlsx")
df_overview = load_excel("vietnam_stocks_overviews.xlsx")
df_price   = load_excel("vietnam_stocks_prices.xlsx")
# df_price['time'] = pd.to_datetime(df_price['time'])
df_price = df_price[df_price['time'] >= '2020-12-31']
# df_cashflow = load_excel("vietnam_stocks_cashflow.xlsx")
# df_income = load_excel("vietnam_stocks_income.xlsx")
df1 = load_excel("ESG_Summary_Assessment Scores_All_Internal_10272025.xlsx")
df2 = load_excel("ESG_Summary_Assessment Scores_All_GICS_11032025.xlsx").rename(columns={'Sector': 'GICS Sector'})
sector_weights = load_excel("sector_weights.xlsx", sheet_name="sector-factor")

In [3]:
# @title
df_raw = load_excel("Ratio_growths & MCAP.xlsx", header=None)
n = 26   # number of columns for df1

df_raw1 = df_raw.iloc[:, 1:n]      # first n columns
df_raw2 = df_raw.iloc[:, [1] + list(range(n, df_raw.shape[1]))]

top1 = df_raw1.iloc[0].ffill(axis=0)
bottom1 = df_raw1.iloc[1].astype(str)
top2 = df_raw2.iloc[0].ffill(axis=0)
bottom2 = df_raw2.iloc[1].astype(str)


df_raw1 = df_raw1.iloc[2:, :]
df_raw1.rename(columns={df_raw1.columns[0]: "Ticker"}, inplace=True)

metrics = top1.iloc[1:]
years = bottom1.iloc[1:]

df_raw1.columns = ["Ticker"] + [f"{m}_{y}" for m, y in zip(metrics, years)]
df_raw1["Ticker"] = df_raw1["Ticker"].astype(str).str.strip()
df_raw1 = df_raw1[df_raw1["Ticker"].ne("") & df_raw1["Ticker"].ne("nan")]

long = df_raw1.melt(id_vars="Ticker", var_name="metric_year", value_name="value")
split = long["metric_year"].str.rsplit("_", n=1, expand=True)
long["metric"] = split[0]
long["year"]   = split[1].str.replace("E", "", regex=False).str.strip()
long["year"] = pd.to_numeric(long["year"], errors="coerce")
long["year"] = long["year"].astype(int)
long["value"]  = pd.to_numeric(long["value"], errors="coerce")
long = long.dropna(subset=["value"])


# --- 6. Pivot to wide panel ---
ratio_growths = (
    long
    .pivot_table(index=["Ticker", "year"], columns="metric", values="value", aggfunc="first")
    .reset_index()
)


df_raw2 = df_raw2.iloc[2:, :]
df_raw2.rename(columns={df_raw2.columns[0]: "Ticker"}, inplace=True)

metrics = top2.iloc[1:]
metrics = metrics.replace("MarketCap", "unadj. MarketCap")
years = bottom2.iloc[1:]

df_raw2.columns = ["Ticker"] + [f"{m}_{y}" for m, y in zip(metrics, years)]
df_raw2["Ticker"] = df_raw2["Ticker"].astype(str).str.strip()
df_raw2 = df_raw2[df_raw2["Ticker"].ne("") & df_raw2["Ticker"].ne("nan")]

long = df_raw2.melt(id_vars="Ticker", var_name="metric_year", value_name="value")
split = long["metric_year"].str.rsplit("_", n=1, expand=True)
long["metric"] = split[0]
long["date"]   = split[1]
long = long.dropna(subset=["value"])

df_mcap = (
    long
    .pivot_table(index=["Ticker", "date"], columns="metric", values="value", aggfunc="first")
    .reset_index())
df_mcap

df_mcap["date"] = pd.to_datetime(df_mcap["date"])
df_mcap["year"] = df_mcap["date"].dt.year
df_mcap["FreeFloat"] = df_mcap["FreeFloat"] / 100
df_mcap["MarketCap"] = df_mcap["FreeFloat"] * df_mcap["unadj. MarketCap"]


# (optional) drop rows where value is missing
df_mcap["MarketCap"] = pd.to_numeric(df_mcap["MarketCap"], errors="coerce")
df_mcap = df_mcap.dropna(subset=["MarketCap"])
df_mcap["half"] = df_mcap["date"].dt.month.map({
    1:1, 2:1, 3:1, 4:1, 5:1, 6:1,
    7:2, 8:2, 9:2, 10:2, 11:2, 12:2})


df_mcap = df_mcap[['Ticker', 'MarketCap', 'year', 'half']]
df_mcap = df_mcap.sort_values(["Ticker", "year", "half"]).reset_index(drop=True)
df_mcap

metric,Ticker,MarketCap,year,half
0,AAA,1134.627841,2020,1
1,AAA,1596.671813,2020,2
2,AAA,2893.409746,2021,1
3,AAA,3296.988410,2021,2
4,AAA,2028.790393,2022,1
...,...,...,...,...
2361,VTZ,86.000000,2023,2
2362,VTZ,269.610000,2024,1
2363,VTZ,458.482195,2024,2
2364,VTZ,450.485412,2025,1


In [4]:
# @title
# Utils
try:
    from google.colab import files
    _IN_COLAB = True
except:  # noqa: E722
    _IN_COLAB = False

def _print_wall_clock(name, t0, timings):
    wall = time.time() - t0
    api_sum = sum(t["seconds"] for t in timings) if timings else 0.0
    avg = (api_sum / len(timings)) if timings else 0.0
    print(f"⏱ {name} — wall-clock: {wall:.2f}s | API total: {api_sum:.2f}s | avg/ticker: {avg:.2f}s over {len(timings)} tickers")

def _prep(df):
    if df is None or df.empty: return df
    if 'year' in df:
        q = df.get('quarter', 1)
        df['_k'] = pd.to_datetime(df['year'].astype(str) + 'Q' + q.astype(str), errors='coerce')
        df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('Int64')
    return df

def _latest_q_last2_a(fetcher):
    q = _prep(fetcher('quarter'))
    a = _prep(fetcher('year'))
    if q is not None and not q.empty: q = q.dropna(subset=['_k']).sort_values('_k').tail(1)
    if a is not None and not a.empty: a = a.dropna(subset=['_k']).sort_values('_k').tail(2)
    parts = [x for x in [a, q] if x is not None and not x.empty]
    return (pd.concat(parts, ignore_index=True).drop(columns=['period','_k'], errors='ignore')
            if parts else pd.DataFrame())

# --- Company overview ---
def fetch_company_overviews(tickers, sleep_secs=0.5, return_timings=False):
    all_data, timings, t0_all = [], [], time.time()
    for i, tk in enumerate(tickers, 1):
        try:
            print(f"📊 {i}/{len(tickers)}: {tk}")
            t0 = time.time()
            ov = Vnstock().stock(symbol=tk, source='TCBS').company.overview()
            timings.append({"function":"company_overview","ticker":tk,"seconds":time.time()-t0})
            df = pd.DataFrame([ov]) if isinstance(ov, dict) else pd.DataFrame(ov)
            df["Ticker"] = tk
            all_data.append(df)
            if sleep_secs: time.sleep(sleep_secs)
        except Exception as e:
            print(f"⚠️ {tk}: {e}")
    _print_wall_clock("Company Overview", t0_all, timings)
    out = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()
    return (out, pd.DataFrame(timings)) if return_timings else out

def fetch_balance_sheets(tickers, lang='vn', sleep_secs=0.5):
    """Fetch last 3–5 annual + latest quarterly balance sheets (year-only), with wall-clock summary."""
    import pandas as pd, time
    rows, n_ok = [], 0
    t0_all = time.time()

    for tk in pd.Series(tickers).dropna().astype(str).unique():
        try:
            s = Vnstock().stock(symbol=tk, source='TCBS')

            # --- Annual (keep as in your original; adjust tail(N) if you truly want 3) ---
            a = s.finance.balance_sheet(period='year', lang=lang, dropna=True)
            a = a.sort_index().tail(5).reset_index().rename(columns={'period':'year'})

            # --- Quarterly (latest only), normalize year column, drop 'quarter' if present ---
            q = s.finance.balance_sheet(period='quarter', lang=lang, dropna=True)
            q = q.sort_index().tail(1).reset_index()
            if 'period' in q.columns:
                y = pd.to_datetime(q['period'], errors='coerce').dt.year
                q['year'] = y.fillna(q['period'].astype(str).str[:4]).astype('Int64')
                q = q.drop(columns=['period'])
            q = q.drop(columns=[c for c in q.columns if c.lower() == 'quarter'], errors='ignore')

            # --- Combine ---
            out = pd.concat([a, q], ignore_index=True)
            out['Ticker'] = tk
            rows.append(out)
            n_ok += 1

            if sleep_secs:
                time.sleep(sleep_secs)

        except Exception as e:
            print(f"⚠️ {tk}: {e}")

    total = time.time() - t0_all
    avg = (total / n_ok) if n_ok else 0.0
    print(f"⏱ Balance Sheet (3Y + latest Q) — Total: {total:.2f}s | Avg per stock: {avg:.2f}s over {n_ok} tickers")

    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()


# --- Income statement: last 2 annual + TTM (year from latest quarter) ---
def fetch_income_statement(tickers, sleep_secs=0.75, lang='en', return_timings=False):
    all_data, timings, t0_all = [], [], time.time()
    for i, tk in enumerate(tickers, 1):
        try:
            print(f"📊 {i}/{len(tickers)}: {tk}")
            s = Vnstock().stock(symbol=tk, source='TCBS')

            # annual (last 2)
            t0 = time.time()
            ya = s.finance.income_statement(period='year', lang=lang, dropna=True)
            timings.append({"function":"income_statement_year","ticker":tk,"seconds":time.time()-t0})
            ya = _prep(ya)
            ya = ya.dropna(subset=['_k']).sort_values('_k').tail(2).drop(columns=['period','_k'], errors='ignore') if ya is not None and not ya.empty else pd.DataFrame()

            # quarterly -> TTM
            t0 = time.time()
            q = s.finance.income_statement(period='quarter', lang=lang, dropna=True)
            timings.append({"function":"income_statement_quarter","ticker":tk,"seconds":time.time()-t0})
            ttm = pd.DataFrame()
            if q is not None and not q.empty:
                q = _prep(q).dropna(subset=['_k']).sort_values('_k').tail(4)
                if not q.empty:
                    num = q.select_dtypes(include='number').columns
                    pct = [c for c in num if ('growth' in c.lower() or 'margin' in c.lower())]
                    sum_cols = [c for c in num if c not in pct]
                    ttm_vals = pd.concat([q[sum_cols].sum(min_count=1), q[pct].mean()])
                    ttm = pd.DataFrame([ttm_vals])
                    # set metadata year = latest quarter's year
                    latest_year = int(q['year'].iloc[-1]) if 'year' in q.columns and pd.notna(q['year'].iloc[-1]) else None
                    if latest_year is not None: ttm['year'] = latest_year

            out = pd.concat([ya, ttm], ignore_index=True)
            if not out.empty: out['Ticker'] = tk; all_data.append(out)
            if sleep_secs: time.sleep(sleep_secs)
        except Exception as e:
            print(f"⚠️ {tk}: {e}")
    _print_wall_clock("Income Statement (year + TTM)", t0_all, timings)
    out = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()
    return (out, pd.DataFrame(timings)) if return_timings else out

# --- Ratios: latest Q + last 2 A, 'year' only ---
def fetch_ratios(tickers, sleep_secs=0.8, lang='en', return_timings=False):
    all_data, timings, t0_all = [], [], time.time()
    for i, tk in enumerate(tickers, 1):
        try:
            print(f"📊 {i}/{len(tickers)}: {tk}")
            s = Vnstock().stock(symbol=tk, source='TCBS')
            t0 = time.time(); df = _latest_q_last2_a(lambda p: s.finance.ratio(period=p, lang=lang, dropna=True)); d = time.time()-t0
            print(f"⏱ {tk} ratios {d:.2f}s"); timings.append({"function":"ratio_combo","ticker":tk,"seconds":d})
            if not df.empty: df['Ticker'] = tk; all_data.append(df)
            if sleep_secs: time.sleep(sleep_secs)
        except Exception as e:
            print(f"⚠️ {tk}: {e}")
    _print_wall_clock("Ratios (latest Q + last 2 A)", t0_all, timings)
    out = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()
    return (out, pd.DataFrame(timings)) if return_timings else out

# --- Cash flow: latest Q + last 2 A, 'year' only ---
def fetch_cashflow(tickers, sleep_secs=0.8, lang='en', return_timings=False):
    all_data, timings, t0_all = [], [], time.time()
    for i, tk in enumerate(tickers, 1):
        try:
            print(f"📊 {i}/{len(tickers)}: {tk}")
            s = Vnstock().stock(symbol=tk, source='TCBS')
            t0 = time.time(); df = _latest_q_last2_a(lambda p: s.finance.cash_flow(period=p, lang=lang, dropna=True)); d = time.time()-t0
            print(f"⏱ {tk} cash_flow {d:.2f}s"); timings.append({"function":"cashflow_combo","ticker":tk,"seconds":d})
            if not df.empty: df['Ticker'] = tk; all_data.append(df)
            if sleep_secs: time.sleep(sleep_secs)
        except Exception as e:
            print(f"⚠️ {tk}: {e}")
    _print_wall_clock("Cash Flow (latest Q + last 2 A)", t0_all, timings)
    out = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()
    return (out, pd.DataFrame(timings)) if return_timings else out

# --- Prices ---
def fetch_prices(tickers, sleep_secs=1.9, start='2022-11-01', end=None, interval='1D'):
    end = end or pd.Timestamp.today().strftime('%Y-%m-%d')
    start_ts = pd.to_datetime(start)
    tickers = pd.Series(tickers).dropna().astype(str).unique().tolist()

    frames, t_all = [], time.time()
    durations = []

    for i, tk in enumerate(tickers, 1):
        t0 = time.time()
        try:
            print(f"📊 {i}/{len(tickers)}: {tk}", end=' ')
            df = Vnstock().stock(symbol=tk, source='TCBS').quote.history(start=start, end=end, interval=interval)
            if df is not None and not df.empty and 'time' in df:
                df = (df.assign(time=pd.to_datetime(df['time'], errors='coerce'))
                        .dropna(subset=['time'])
                        .loc[lambda x: x['time'] >= start_ts, ['time','close']]
                        .set_index('time')
                        .rename(columns={'close': tk}))
                frames.append(df[~df.index.duplicated(keep='last')].sort_index())
            d = time.time() - t0
            durations.append(d)
            print(f"⏱ {d:.2f}s")
        except Exception as e:
            print(f"⚠️ {tk}: {e}")
        if sleep_secs: time.sleep(sleep_secs)

    total = time.time() - t_all
    avg = total / len(tickers) if tickers else 0
    print(f"\n⏱ Total: {total:.2f}s | Avg per stock: {avg:.2f}s over {len(tickers)} tickers")

    return pd.concat(frames, axis=1) if frames else pd.DataFrame()

# --- Export ---
def export_to_excel(df, filename, auto_download=True):
    df.to_excel(filename, index=True)
    print(f"💾 Saved {filename} ({df.shape})")
    if auto_download and _IN_COLAB:
        files.download(filename)

In [5]:
# @title
tickers = df_mcap['Ticker'].dropna().unique().tolist()
start = '2020-01-01'

# income = fetch_income_statement(tickers)
# income
# cashflow = fetch_cashflow(tickers, period='quarter', latest_only=True)
# cashflow
# balance_sheets = fetch_balance_sheets(tickers, sleep_secs=2.5)
# balance_sheets
# ratios = fetch_ratios(tickers)
# ratios
# prices = fetch_prices(tickers, sleep_secs=6, start=start, end=None, interval='1D')
# prices
# overviews = fetch_company_overviews(tickers)
# overviews

# export_to_excel(overviews, "vietnam_stocks_overviews.xlsx", auto_download=True)

In [10]:
from vnstock import Quote
quote = Quote(symbol='VNINDEX', source='VCI')
quote.history(start='2020-01-01', end='2024-05-25')

RetryError: RetryError[<Future at 0x7bd7a006c3b0 state=finished raised ConnectionError>]

In [7]:
# @title
vn_raw = Vnstock().stock('VNINDEX', source='TCBS').quote.history(
    start=start, end=pd.Timestamp.today().strftime('%Y-%m-%d'), interval='1D')[['time','close']]


vnindex = (vn_raw
    .drop_duplicates('time', keep='last')
    .assign(Date=lambda df: pd.to_datetime(df['time']))
    .sort_values('Date')
    .assign(DailyReturn=lambda df: df['close'].pct_change())
    .set_index('Date')[['DailyReturn']]
    .dropna()
)


def load_csv(filename):
    try:
        return pd.read_csv(filename, encoding='utf-8-sig')
    except:
        print(f"--- Import file: {filename} ---")
        files.upload()
        return pd.read_csv(filename, encoding='utf-8-sig')


# vnindex = load_csv('VN100 Historical Data.csv') #Download from this website: https://www.investing.com/indices/vn100-historical-data
# vnindex['Date'] = pd.to_datetime(vnindex['Date'], errors='coerce')
# vnindex = vnindex.sort_values('Date')
# vnindex['Price'] = (vnindex['Price'].astype(str).str.replace(',', '', regex=False)
#       .str.replace(r'[^0-9.\-]', '', regex=True)  # drop non-numeric (just in case)
#       .replace({'': None, '-': None}))
# vnindex['Price'] = pd.to_numeric(vnindex['Price'], errors='coerce')
# vnindex['DailyReturn'] = vnindex['Price'].pct_change()  # simple percentage return
# vnindex = vnindex.set_index('Date')
# vnindex = vnindex[['DailyReturn']]
# vnindex = vnindex.dropna()

# rf_df = load_csv('Vietnam 10-Year Bond Yield Historical Data.csv') #Download from this website: https://www.investing.com/rates-bonds/vietnam-10-year-bond-yield-historical-data
# rf_df['Date'] = pd.to_datetime(rf_df['Date'])
# rf_df = rf_df.sort_values('Date')
# rf_df['rf'] = (rf_df['Price'] / 100) / 252   # assuming 252 trading days per year
# rf_df = rf_df.set_index('Date')[['rf']]

2025-12-15 05:22:11 - vnstock.common.data - INFO - TCBS listing data fallback to VCI
INFO:vnstock.common.data:TCBS listing data fallback to VCI
2025-12-15 05:22:12 - vnstock.explorer.tcbs.quote - ERROR - Dữ liệu không tồn tại từ 2020-01-01 đến 2020-12-31: Tải dữ liệu không thành công: 404 - Not Found
ERROR:vnstock.explorer.tcbs.quote:Dữ liệu không tồn tại từ 2020-01-01 đến 2020-12-31: Tải dữ liệu không thành công: 404 - Not Found
2025-12-15 05:22:13 - vnstock.explorer.tcbs.quote - ERROR - Dữ liệu không tồn tại từ 2021-01-01 đến 2021-12-31: Tải dữ liệu không thành công: 404 - Not Found
ERROR:vnstock.explorer.tcbs.quote:Dữ liệu không tồn tại từ 2021-01-01 đến 2021-12-31: Tải dữ liệu không thành công: 404 - Not Found
2025-12-15 05:22:13 - vnstock.explorer.tcbs.quote - ERROR - Dữ liệu không tồn tại từ 2022-01-01 đến 2022-12-31: Tải dữ liệu không thành công: 404 - Not Found
ERROR:vnstock.explorer.tcbs.quote:Dữ liệu không tồn tại từ 2022-01-01 đến 2022-12-31: Tải dữ liệu không thành công: 40

RetryError: RetryError[<Future at 0x7bd7a00499d0 state=finished raised ValueError>]

## 1.2 Data Cleaning


In [ ]:
# @title
merged = pd.merge(df1, df2[['Ticker', 'GICS Sector']], on='Ticker', how='left')  # or 'left', 'right', 'outer'
merged = pd.merge(merged, df_overview[['Ticker', 'industry']], on='Ticker', how='left')  # or 'left', 'right', 'outer'
merged.loc[merged['GICS Sector'] == 'Not Classified', 'GICS Sector'] = (merged.loc[merged['GICS Sector'] == 'Not Classified', 'industry'])

recode = {
    'Food & Beverage': 'Food, Beverage & Tobacco',
    'Real Estate': 'Real Estate Management & Development',
    'Telecommunications': 'Telecommunication Services',
    'Construction & Materials': 'Materials', 'Chemicals': 'Materials',
    'Oil & Gas': 'Transportation',
    'Industrial Goods & Services': 'Capital Goods',
    'Travel & Leisure': 'Consumer Services'}
merged['GICS Sector'] = merged['GICS Sector'].replace(recode)

merged = merged.drop(columns=['No', 'Form', 'industry'])
# merged = pd.merge(merged, sector_weights.drop(columns=['No.']), left_on='GICS Sector', right_on='GICS Level 2 Sector', how='left').drop(columns=['GICS Level 2 Sector'])
merged = merged.drop_duplicates(subset='Ticker')

In [ ]:
# @title
df = merged.copy()

# ---- ESG columns ----
grade_map  = {'A': 5, 'B': 4, 'C': 3, 'D': 2, 'E': 1}
pseudo_map = {'A': 90, 'B': 70, 'C': 50, 'D': 30, 'E': 0}

df['E&S Rate Recoded']   = df['E&S Rate'].map(grade_map)
df['Pseudo E&S score']   = df['E&S Rate'].map(pseudo_map)
# df['Consolidated ESG score'] = (
#     df['Pseudo E&S score'] * df['E&S Factor (%)']
#     + df['CG Factor (%)'] * df['CG Rate (%)']
# ).round(1)

# ---- Sector mapping + overrides ----
sector_map = {
    'Bank':'Financials','Insurance':'Financials','Brokerage':'Financials',
    'Residential':'Real Estate & Construction','IP':'Real Estate & Construction',
    'Chemicals':'Material, Energy & Utility','Metals':'Material, Energy & Utility',
    'Energy':'Material, Energy & Utility','Industrials':'Material, Energy & Utility',
    'Utilities':'Material, Energy & Utility','Other Materials':'Material, Energy & Utility',
    'Retailing':'Retail',
    'Not Classified':'Others','IT':'Others','Transportation':'Others','Conglomerate':'Others',
    'Telecom & Media':'Others','Health Care':'Others',
    'Consumer Durables':'Others','Consumer Staples':'Others'
}

manual_override = {
    'HDG':'Real Estate & Construction','HBC':'Real Estate & Construction',
    'C4G':'Real Estate & Construction','HHV':'Real Estate & Construction',
    'DDV':'Material, Energy & Utility','CTD':'Real Estate & Construction',
    'HHS':'Real Estate & Construction','FCN':'Real Estate & Construction',
    'EVG':'Real Estate & Construction','LCG':'Real Estate & Construction',
    'CRV':'Real Estate & Construction','VLB':'Material, Energy & Utility',
    'ASM':'Others','CTR':'Retail','VC3':'Real Estate & Construction'
}

df['Sector'] = df['Sector'].replace(sector_map)
df['Sector'] = df['Ticker'].map(manual_override).fillna(df['Sector'])

# -----------------------------
# 2. Create sector_map table
# -----------------------------
sector_map_df = (
    pd.DataFrame(sector_map.items(), columns=['Old', 'New'])
    .groupby('New')['Old']
    .apply(lambda lst: ", ".join(lst))   # remove brackets
    .reset_index()
)

print("=== Sector Mapping Table ===")
display(sector_map_df)

# -----------------------------
# 3. Create manual override table
# -----------------------------
override_df = (
    pd.DataFrame(manual_override.items(), columns=['Ticker', 'New'])
    .groupby('New')['Ticker']
    .apply(lambda lst: ", ".join(lst))   # remove brackets
    .reset_index()
)
print("\n\n === Manual Override Table ===")
display(override_df)

In [ ]:
# @title
# === 3. Row-level table (unchanged) ===
df_tmp = merged.copy()
df_tmp['Sector_old']    = df_tmp['Sector']
df_tmp['Sector_mapped'] = df_tmp['Sector_old'].replace(sector_map)
df_tmp['Sector_final']  = df_tmp.apply(
    lambda r: manual_override.get(r['Ticker'], r['Sector_mapped']), axis=1
)

row_level_table = df_tmp[['Ticker','Sector_old','Sector_mapped','Sector_final']].drop_duplicates()
display(row_level_table)

In [ ]:
# @title
df_long = (df_price.melt(id_vars='time', var_name='Ticker', value_name='Price').rename(columns={'time': 'Date'}))
# df_long = df_long.merge(df.drop(columns=['E&S Factor (%)', 'CG Factor (%)', 'GICS Sector']), on='Ticker', how='left')
# df_long = df_long.merge(df_overview[['Ticker', 'outstanding_share']], on='Ticker', how='left')
df_long = df_long.drop_duplicates(subset=['Date', 'Ticker'])
df_long['DailyReturn'] = (df_long.groupby('Ticker')['Price'].pct_change())

df_long['Date'] = pd.to_datetime(df_long['Date'])
df_long = df_long.sort_values(['Ticker', 'Date'])
df_long['year'] = df_long['Date'].dt.year
df_long['quarter'] = df_long['Date'].dt.quarter
df_long["half"] = df_long["Date"].dt.month.map({
    1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1,   # H1 months
    7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2})
df_long = df_long.merge(df_mcap, on=['Ticker', 'half', 'year'], how='left')

## NEW Marketcap
df_long_new = df_long.copy()
mc_half = (df_long_new.groupby(['Ticker', 'year', 'half'], as_index=False)['MarketCap']
           .first().sort_values(['Ticker', 'year', 'half']))
mc_half['MCapLag'] = mc_half.groupby('Ticker')['MarketCap'].shift(1)
df_long_new = df_long_new.merge(
    mc_half[['Ticker', 'year', 'half', 'MCapLag']],
    on=['Ticker', 'year', 'half'],how='left')
df_market_cap = (df_long_new.groupby(['Date','Ticker'])['MCapLag'].last().unstack().sort_index())

# Use LAGGED market cap for weights (prev day per ticker)
# last_price = (df_long.sort_values("Date").groupby(["Ticker", 'year', "half"])["Price"]
#            .last().rename("LastPriceHY").reset_index())
# df_long = df_long.merge(last_price, on=["Ticker", 'year', "half"], how="left")
# df_long['MarketCap'] = df_long['LastPriceHY'] * df_long['outstanding_share']

# mc_half = (df_long.groupby(['Ticker', 'year', 'half'], as_index=False)['MarketCap']
#            .first().sort_values(['Ticker', 'year', 'half']))
# mc_half['MCapLag'] = mc_half.groupby('Ticker')['MarketCap'].shift(1)
# df_long = df_long.merge(
#     mc_half[['Ticker', 'year', 'half', 'MCapLag']],
#     on=['Ticker', 'year', 'half'],how='left')

# df_market_cap = (df_long.groupby(['Date','Ticker'])['MCapLag'].last().unstack().sort_index())
df_returns = df_long.pivot(index='Date', columns='Ticker', values='DailyReturn').sort_index()

latest_tags = df[['Ticker','E&S Rate']]
grades_order = ['A','B','C','D','E']
present_grades = [g for g in grades_order if g in df['E&S Rate'].unique()]
group_tickers = {g: latest_tags.loc[latest_tags['E&S Rate'] == g, 'Ticker'].tolist() for g in present_grades}

# df_long

In [ ]:
# @title
# ---- Build sector × grade ticker table ----
target_sectors = df['Sector'].unique()
target_grades = ['A', 'B', 'C', 'D', 'E']

# unique sorted tickers per (Sector, Grade)
sg = (df[df['Sector'].isin(target_sectors)]
      .groupby(['Sector', 'E&S Rate'])['Ticker']
      .apply(lambda s: sorted(set(s)))
      .unstack(fill_value=[])
      .reindex(index=target_sectors, columns=target_grades))

def wrap_tickers(tickers, n=6, indent="    "):
    if not tickers:
        return ""
    return "\n".join(
        (indent if i else "") + ", ".join(tickers[i:i+n])
        for i in range(0, len(tickers), n)
    )

sector_grade_df = sg.applymap(wrap_tickers)
sector_grade_df.index.name = "Sector"

# display nicely in notebook
display(
    sector_grade_df.style.set_properties(**{
        "white-space": "pre-wrap",
        "vertical-align": "top",
        "font-family": "monospace",
    })
)

# ---- Export to Excel ----
# excel_df = sector_grade_df.applymap(
#     lambda x: x.replace("\n", "\r\n") if isinstance(x, str) else x
# )
# excel_df.to_excel("sector_grade_table.xlsx", sheet_name="Sector_Grade", index=True)

In [ ]:
# df_mcap_merged = df_market_cap.merge(df_mcap_new, on="Date", how="left")
# df_mcap_merged = df_mcap_merged[sorted(df_mcap_merged.columns, reverse=False)]
# df_mcap_merged = df_mcap_merged.drop_duplicates(subset=['AAA_x', 'AAA_y'])
# df_mcap_merged

# 2.Introduction

In [ ]:
# # @title
# # Drop missing ESG scores
# data = df.dropna(subset=['Consolidated ESG score'])

# # Select top 5 sectors by frequency
# top_sectors = data['Sector'].value_counts().head(5).index

# # Prepare data for boxplots
# box_data = [data.loc[data['Sector'] == s, 'Consolidated ESG score'].dropna() for s in top_sectors]

# # Create figure with 1 row, 2 columns (left = histogram, right = stacked boxplots)
# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5), gridspec_kw={'width_ratios': [1.2, 1]})

# # ================== 1️⃣ Left: Distribution ==================
# ax = axes[0]
# ax.hist(data['Consolidated ESG score'], bins=range(0, 101, 2), edgecolor='black', alpha=0.7)
# mean_val = data['Consolidated ESG score'].mean()
# ax.axvline(mean_val, color='red', linestyle='--', label=f'Mean: {mean_val:.2f}')

# ax.set_xlim(0, 100)
# ax.set_title('Overall Distribution', fontsize=12)
# ax.set_xlabel('Consolidated ESG Score')
# ax.set_ylabel('Frequency')
# ax.legend(fontsize=8)
# ax.grid(alpha=0.3)

# # ================== 2️⃣ Right: Horizontal boxplots (stacked) ==================
# ax2 = axes[1]
# ax2.boxplot(box_data, vert=False, patch_artist=True, labels=top_sectors)
# ax2.set_xlim(0, 100)
# ax2.set_title('Consolidated ESG Score by Top 5 Sectors', fontsize=12)
# ax2.set_xlabel('Consolidated ESG Score')
# ax2.grid(alpha=0.3)

# # ================== Final layout ==================
# fig.suptitle('Consolidated ESG Score — Overall Distribution and Sector Comparison', fontsize=15)
# plt.tight_layout(rect=[0, 0, 1, 0.95])
# plt.show()

In [ ]:
# @title
# --- cap schedule ---
cap_by_n = {
    1: 1, 2: 0.60, 3: 0.50, 4: 0.40, 5: 0.35, 6: 0.30, 7: 0.28, 8: 0.26, 9: 0.24, 10: 0.22
}
def get_cap(n):
    return cap_by_n[max(2, min(int(n), 10))]


def cap_weights_row(w):
    """
    Apply dynamic cap based on number of constituents (n),
    iteratively reassign overflow until all w <= cap and sum(w)=1.
    """
    w_raw = w.fillna(0).clip(lower=0)
    if w_raw.sum() == 0:
        return w_raw

    active = w_raw > 0
    cap = get_cap(active.sum())
    w_adj = pd.Series(0.0, index=w.index)
    remaining = 1.0

    for _ in range(len(w)):
        # allocate remaining proportionally among active names
        alloc = w_raw[active] * (remaining / w_raw[active].sum())
        w_adj[active] = alloc

        # detect breaches
        over = (w_adj > cap) & active
        if not over.any():
            break

        # fix those at cap
        w_adj[over] = cap
        active[over] = False
        remaining = 1 - w_adj[~active].sum()

    return w_adj

In [ ]:
# @title
# ------------------------------------------------------------
# 0) Define High ESG tickers (score > 70)
# ------------------------------------------------------------
# high_esg_tickers = (
#     df.loc[df['Consolidated ESG score'] > 70, 'Ticker']
#       .dropna()
#       .unique()
#       .tolist()
)
# print(f"High ESG portfolio has {len(high_esg_tickers)} stocks.")
# display(df[df['Ticker'].isin(high_esg_tickers)].drop(columns=['E&S Rate Recoded']).sort_values('Consolidated ESG score', ascending=False))

# ------------------------------------------------------------
# 1) MARKET-CAP WEIGHTED
# ------------------------------------------------------------
common = df_returns.index.intersection(vnindex.index).sort_values()
df_returns = df_returns.reindex(common)
df_market_cap = df_market_cap.reindex(common)
vnindex = vnindex.reindex(common)

es_mw_portfolio = pd.DataFrame(index=common)
exclude = {"VIC"} #, 'HPG', 'VHM', 'NVL', 'EIB', 'LPB'

# A–E buckets from your grouping
for grade in group_tickers:
    valid = [
        t for t in group_tickers[grade]
        if t not in exclude
        and t in df_returns.columns
        and t in df_market_cap.columns]
    if not valid:
        continue
    R = df_returns[valid]
    MC = df_market_cap[valid]
    w_raw = MC.divide(MC.sum(axis=1).replace(0, np.nan), axis=0) # Step 1: raw MC weights
    # w_raw_new = df_mcap_new[valid].divide(df_mcap_new[valid].sum(axis=1).replace(0, np.nan), axis=0) # Step 1: raw MC weights
    # w_capped = w_raw.apply(cap_weights_row, axis=1) # Step 2: apply capping row-wise (per date)
    es_mw_portfolio[grade] = (w_raw * R).sum(axis=1) # Step 3: portfolio return

    # w_merged = w_raw.merge(w_raw_new, on="Date", how="left")
    # w_merged = w_merged[sorted(w_merged.columns)]
    print(grade)
    w_raw = w_raw.drop_duplicates(subset=w_raw.columns[0], keep='first')
    display(w_raw*100)

    # filename = f"w_raw_{grade}.xlsx"
    # w_raw.to_excel(filename, index=True)   # <-- correct writer for Excel
    # files.download(filename)

# High ESG (MC-weighted)
# valid_high_mw = [t for t in high_esg_tickers if t in df_returns.columns and t in df_market_cap.columns]
# if valid_high_mw:
#     R = df_returns[valid_high_mw]
#     MC = df_market_cap[valid_high_mw]
#     w = MC.divide(MC.sum(axis=1).replace(0, np.nan), axis=0)
#     es_mw_portfolio['High ESG'] = (w * R).sum(axis=1)

# Add VN Index and tidy
es_mw_portfolio = es_mw_portfolio.join(
    vnindex[['DailyReturn']].rename(columns={'DailyReturn':'VN Index'}),
    how='left'
)

# Keep columns that exist (A–E, High ESG, VN Index) and coerce numerics
mw_cols_order = [c for c in ['A','B','C','D','E','VN Index'] if c in es_mw_portfolio.columns]
es_mw_portfolio = es_mw_portfolio.apply(pd.to_numeric, errors='coerce')[mw_cols_order]

# Ensure the first day is 0 return
if len(es_mw_portfolio) > 0:
    es_mw_portfolio.iloc[0] = 0.0

# Cumulate
es_mw_portfolio = es_mw_portfolio[~es_mw_portfolio.index.duplicated(keep='first')]
es_mw_cum = (1 + es_mw_portfolio).cumprod() - 1

# ------------------------------------------------------------
# 2) EQUAL-WEIGHTED
# ------------------------------------------------------------
ew_portfolio = pd.DataFrame(index=common)

# A–E buckets
for grade, tickers in group_tickers.items():
    valid = [t for t in tickers if t in df_returns.columns and t not in exclude]
    if not valid:
        continue
    ew_portfolio[grade] = df_returns[valid].mean(axis=1, skipna=True)

# High ESG (EW)
# valid_high = [t for t in high_esg_tickers if t not in exclude
                                        #  and t in df_returns.columns]
# if valid_high:
#     ew_portfolio['High ESG'] = df_returns[valid_high].mean(axis=1, skipna=True)

# Add VN Index and tidy
ew_portfolio = ew_portfolio.join(
    vnindex[['DailyReturn']].rename(columns={'DailyReturn':'VN Index'}),
    how='left'
)

ew_cols_order = [c for c in ['A','B','C','D','E','VN Index'] if c in ew_portfolio.columns]
ew_portfolio = ew_portfolio.apply(pd.to_numeric, errors='coerce')[ew_cols_order]

if len(ew_portfolio) > 0:
    ew_portfolio.iloc[0] = 0.0

ew_portfolio = ew_portfolio[~ew_portfolio.index.duplicated(keep='first')]
ew_cum = (1 + ew_portfolio).cumprod() - 1.0

In [ ]:
# @title
# ------------------------------------------------------------
# 3) PLOTTING
# ------------------------------------------------------------
custom_order = ['E&S A', 'E&S B', 'E&S C', 'E&S D', 'E&S E', 'High ESG', 'VN Index']
es_cols = set(group_tickers.keys())  # {'A','B','C','D','E'}
liberation_day = dt.datetime(2025, 4, 2)

# --- Count number of stocks per group ---
# Assumes group_tickers = {'A': [...], 'B': [...], ..., 'E': [...]}.
# If you also have tickers for 'High ESG', add them there and it will be picked up.
group_sizes = {g: len([t for t in tks if t not in exclude])
    for g, tks in group_tickers.items()}

# --- Softer color palette (lighter tones) ---
color_map = {
    'A': 'green',
    'B': 'gold',
    'C': '#B85400',        # yellow/gold
    'D': 'red',
    'E': 'black',
    # 'High ESG': 'darkgreen',
    'VN Index': 'purple'
}

def plot_panel(ax, cum_df, title):
    handles = {}
    label_by_col = {}

    # ---- Plot each cumulative return series ----
    for col in cum_df.columns:

        # Build legend label with N
        if col in es_cols:
            n = group_sizes.get(col)
            label = f'E&S {col} (N={n})' if n is not None else f'E&S {col}'
        elif col == 'High ESG':
            n = group_sizes.get('High ESG')
            label = f'High ESG (N={n})' if n is not None else 'High ESG'
        else:
            label = col

        color = color_map.get(col, 'gray')
        ls = '--' if col == 'VN Index' else '-'
        lw = 1.3 if col == 'VN Index' else 1.0

        line, = ax.plot(
            cum_df.index,
            cum_df[col],
            label=label,
            linestyle=ls,
            linewidth=lw,
            color=color
        )

        handles[col] = line
        label_by_col[col] = label

    # ---- Legend in custom order ----
    legend_order_cols = ['A', 'B', 'C', 'D', 'E', 'High ESG', 'VN Index']
    ordered_cols = [c for c in legend_order_cols if c in handles]

    ordered_handles = [handles[c] for c in ordered_cols]
    ordered_labels = [label_by_col[c] for c in ordered_cols]

    ax.legend(ordered_handles, ordered_labels, loc='best', fontsize=9)

    # ---- Titles, labels, grid ----
    ax.set_title(title)
    ax.set_ylabel('Cumulative Return (%)')
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    ax.grid(True, alpha=0.3)

    # ---- X-axis tick formatting ----
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.xaxis.set_major_locator(mdates.MonthLocator())

    plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
    ax.tick_params(axis='x', pad=10)   # <-- pull month labels downward

    # ---- X-limits ----
    xmin, xmax = cum_df.index.min(), cum_df.index.max()
    ax.set_xlim(xmin, xmax)

    # ---- Year lines + labels below plot ----
    years = range(xmin.year, xmax.year + 1)

    for y in years:
        xs = pd.Timestamp(f'{y}-01-01')
        if xs < xmin or xs > xmax:
            continue

        # Vertical year line
        ax.axvline(xs, color='gray', linestyle=':', linewidth=0.6)

        # Year label placed BELOW month labels
        ax.text(
            pd.Timestamp(f'{y}-07-01'),
            -0.02,               # lower than x-ticks (axes coords)
            str(y),
            ha='center',
            va='top',
            fontsize=9,
            color='gray',
            transform=ax.get_xaxis_transform()   # x = data, y = axes coords
        )

# Keep consistent column order
plot_cols = [c for c in ['A','B','C','D','E','High ESG','VN Index']
             if c in ew_cum.columns or c in es_mw_cum.columns]

ew_cum_plot = ew_cum[[c for c in plot_cols if c in ew_cum.columns]]
mw_cum_plot = es_mw_cum[[c for c in plot_cols if c in es_mw_cum.columns]]

# --- Figure layout ---
fig, axes = plt.subplots(2, 2, figsize=(17, 9), gridspec_kw={'width_ratios': [3.5, 1]})

# Left column: cumulative return plots
plot_panel(axes[0, 0], ew_cum_plot, 'Cumulative Returns — Equal-Weighted (exc. VIC)')
plot_panel(axes[1, 0], mw_cum_plot, 'Cumulative Returns — Free Float Market-Cap-Weighted (exc. VIC)')

# Right column: correlation heatmaps
ew_corr_cols = [c for c in ['A','B','C','D','E','High ESG','VN Index'] if c in ew_portfolio.columns]
ew_corr = ew_portfolio[ew_corr_cols].corr()
sns.heatmap(
    ew_corr,
    ax=axes[0, 1],
    annot=True,
    cmap='RdYlBu',
    center=0,
    fmt=".2f",
    cbar=True,
    square=True
)
axes[0, 1].set_title('Correlation — Equal-Weighted exc. VIC')

mw_corr_cols = [c for c in ['A','B','C','D','E','High ESG','VN Index'] if c in es_mw_portfolio.columns]
mw_corr = es_mw_portfolio[mw_corr_cols].corr()
sns.heatmap(
    mw_corr,
    ax=axes[1, 1],
    annot=True,
    cmap='RdYlBu',
    center=0,
    fmt=".2f",
    cbar=True,
    square=True
)
axes[1, 1].set_title('Correlation — Mkt-Cap-Weighted exc. VIC')

print(
    'These figures imply that poorly rated ESG companies (E) behave more idiosyncratically — '
    'their returns are less aligned with the market, while High ESG firms (>70 score) show '
    'distinct performance patterns relative to the VN Index.'
)

plt.tight_layout()
plt.show()

In [ ]:
#@title
fig2, ax2 = plt.subplots(figsize=(15, 5))
plot_panel(ax2, mw_cum_plot, 'Cumulative Returns — Free Float Market-Cap-Weighted (exc. VIC)')
plt.subplots_adjust(bottom=0.2)
plt.show()

In [ ]:
# @title
# ------------------------------------------------------------
# 3) YEARLY GROUPED BAR CHART (MARKET-CAP WEIGHTED ONLY)
# ------------------------------------------------------------
from matplotlib.ticker import PercentFormatter

# 1) Calendar-year returns from daily simple returns
annual_ret = (1 + es_mw_portfolio).groupby(es_mw_portfolio.index.year).prod() - 1
annual_ret.index.name = "Year"

# 2) Keep relevant columns: A–E + VN Index (only those that exist)
cols = [c for c in ['A', 'B', 'C', 'D', 'E', 'VN Index'] if c in annual_ret.columns]
annual_ret = annual_ret[cols]

# 3) Drop first year if all returns are exactly zero
if len(annual_ret) > 0:
    first_year = annual_ret.index[0]
    if (annual_ret.loc[first_year].fillna(0).abs().sum() == 0):
        annual_ret = annual_ret.drop(first_year)

# If nothing left, bail out gracefully
if annual_ret.empty:
    print("No non-zero annual returns to plot.")
else:
    years = annual_ret.index.to_numpy()
    n_years = len(years)
    n_series = len(cols)

    x = np.arange(n_years)  # base x positions for each year
    bar_width = 0.8 / n_series  # total width 0.8, split across series

    fig, ax = plt.subplots(figsize=(12, 5))

    for i, col in enumerate(cols):
        # Offsets so the bars for each label sit side-by-side within each year
        offsets = x - 0.4 + bar_width/2 + i * bar_width

        data = annual_ret[col].values
        label = f"E&S {col}" if col in ['A','B','C','D','E'] else "VN Index"
        color = color_map.get(col, 'gray')  # reuse your color_map

        ax.bar(
            offsets,
            data,
            width=bar_width,
            label=label,
            color=color
        )

    # X-axis: main ticks at the center of each year-group
    ax.set_xticks(x)
    ax.set_xticklabels(years)
    ax.set_xlabel("Year")

    # Y-axis formatting
    ax.yaxis.set_major_formatter(PercentFormatter(1.0))
    ax.set_ylabel("Yearly Return")

    ax.set_title("Free Float Market-Cap-Weighted — Yearly Returns by E&S Bucket and VN Index (exc. VIC)")
    ax.grid(axis='y', alpha=0.3)

    # Legend = "sub x axis" (labels within each year)
    ax.legend(title="Portfolio", ncol=min(n_series, 4))

    # Optional: reasonable y-limits based on all data
    ymin = min(0, np.nanmin(annual_ret.values) * 1.2)
    ymax = max(0, np.nanmax(annual_ret.values) * 1.2)
    ax.set_ylim(ymin, ymax)

    plt.tight_layout()
    plt.show()


In [ ]:
# @title
# -----------------------------------
# Settings
# -----------------------------------
sector_col = "Sector"
target_sectors = ["Retail", "Financials", "Real Estate & Construction", "Material, Energy & Utility", "Others"]
target_grades = ["A", "B", "C", "D", "E"]

pd.set_option("display.max_colwidth", None)

sector_mw_cum = {}
sector_group_sizes = {}
sector_grade_table = {sec: {g: [] for g in target_grades} for sec in target_sectors}

# -----------------------------------
# 1. Your original sector calculations + collect tickers
# -----------------------------------
for sec in target_sectors:

    tickers_sec = (
        df.loc[df[sector_col] == sec, "Ticker"]
        .dropna().unique()
    )

    if len(tickers_sec) == 0:
        continue

    sec_port = pd.DataFrame(index=common)
    sec_sizes = {}

    for grade, group in group_tickers.items():

        valid = [
            t for t in group
            if t in tickers_sec
            and t in df_returns.columns
            and t in df_market_cap.columns
        ]

        if not valid:
            continue

        # record tickers for the table
        sector_grade_table[sec][grade] = valid

        # your return calculations (unchanged)
        R = df_returns[valid]
        MC = df_market_cap[valid]
        w_raw = MC.div(MC.sum(axis=1), axis=0)

        sec_port[grade] = (w_raw * R).sum(axis=1)
        sec_sizes[grade] = len(valid)

    sector_group_sizes[sec] = sec_sizes

    sec_port["VN Index"] = vnindex["DailyReturn"]

    cols = [c for c in ["A", "B", "C", "D", "E", "VN Index"] if c in sec_port.columns]
    sec_port = sec_port[cols].apply(pd.to_numeric, errors="coerce")

    if len(sec_port) > 0:
        sec_port.iloc[0] = 0

    sec_port = sec_port[~sec_port.index.duplicated(keep="first")]
    sector_mw_cum[sec] = (1 + sec_port).cumprod() - 1

In [ ]:
# @title
def plot_panel(ax, cum_df, title, group_sizes_override=None):
    handles = {}
    label_by_col = {}

    # choose which group_sizes to use
    local_group_sizes = group_sizes_override if group_sizes_override is not None else group_sizes

    for col in cum_df.columns:
        # label with N where available
        if col in es_cols:
            n = None if local_group_sizes is None else local_group_sizes.get(col)
            if n is not None:
                label = f'E&S {col} (N={n})'
            else:
                label = f'E&S {col}'
        elif col == 'High ESG':
            n = None if local_group_sizes is None else local_group_sizes.get('High ESG')
            label = f'High ESG (N={n})' if n is not None else 'High ESG'
        else:
            label = col

        color = color_map.get(col, 'gray')
        ls = '--' if col == 'VN Index' else '-'
        lw = 1.3 if col == 'VN Index' else 1.0

        line, = ax.plot(
            cum_df.index,
            cum_df[col],
            label=label,
            linestyle=ls,
            linewidth=lw,
            color=color
        )

        handles[col] = line
        label_by_col[col] = label

    # legend in fixed order
    legend_order_cols = ['A', 'B', 'C', 'D', 'E', 'High ESG', 'VN Index']
    ordered_cols = [c for c in legend_order_cols if c in handles]
    ordered_handles = [handles[c] for c in ordered_cols]
    ordered_labels = [label_by_col[c] for c in ordered_cols]

    ax.legend(ordered_handles, ordered_labels, loc='best', fontsize=9)
    ax.set_title(title)
    ax.set_ylabel('Cumulative Return (%)')
    ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
    ax.grid(True, alpha=0.3)

    # ----- x-axis: months + rotate + lower tick labels -----
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
    ax.tick_params(axis='x', pad=10)   # move month labels downward

    # ----- x-limits -----
    xmin, xmax = cum_df.index.min(), cum_df.index.max()
    ax.set_xlim(xmin, xmax)

    # ----- year lines + labels below months -----
    years = range(xmin.year, xmax.year + 1)
    for y in years:
        xs = pd.Timestamp(f'{y}-01-01')
        if xs < xmin or xs > xmax:
            continue

        # vertical line at Jan 1
        ax.axvline(xs, color='gray', linestyle=':', linewidth=0.6)

        # year label below the month ticks
        ax.text(
            pd.Timestamp(f'{y}-07-01'),
            -0.02,                # below ticks (axes coords)
            str(y),
            ha='center',
            va='top',
            fontsize=9,
            color='gray',
            transform=ax.get_xaxis_transform()   # x=data, y=axes coords
        )


In [ ]:
# fig, axes = plt.subplots(4, 1, figsize=(15, 16), sharex=True)

# for ax, sec in zip(axes, target_sectors):
#     df_sec = sector_mw_cum.get(sec)
#     if df_sec is None or df_sec.empty:
#         ax.set_visible(False)
#         continue

#     # count # of tickers in this sector
#     n_stocks = df[df['Sector'] == sec]['Ticker'].nunique()

#     cols = [c for c in ["A","B","C","D","D ex VIC","E","VN Index"] if c in df_sec.columns]

#     plot_panel(
#         ax,
#         df_sec[cols],
#         f"{sec} — Cumulative Returns (MC-weighted,  N={n_stocks}"
#     )

# plt.tight_layout()
# plt.show()

# @title
for sec in target_sectors:
    df_sec = sector_mw_cum.get(sec)
    if df_sec is None or df_sec.empty:
        continue

    n_stocks = df[df['Sector'] == sec]['Ticker'].nunique()
    cols = [c for c in ["A","B","C","D","E","VN Index"] if c in df_sec.columns]

    fig, ax = plt.subplots(figsize=(14, 4.5))
    plot_panel(
        ax,
        df_sec[cols],
        f"{sec} — Cumulative Returns (Free Float MC-weighted,  N={n_stocks})",
        group_sizes_override=sector_group_sizes.get(sec, {})  # sector-specific Ns
    )
    plt.tight_layout()
    plt.show()
